In [2]:
from Bio import Entrez
from Bio import SeqIO

genes_int = ["mprF","ileS"]
local = {}
gene_seq = {}
prot_id = {}
prot = {}

Entrez.email = "pg45476@alunos.uminho.pt"
handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="CP000253.1")
seq_record = SeqIO.read(handle, "genbank")

################################
print("Título:",seq_record.description)
print("ID GenBank (e outros IDs):",seq_record.annotations["accessions"])
print("Taxonomia:",seq_record.annotations["taxonomy"])
print("Tipo de molécula:",seq_record.annotations["molecule_type"])
print("Topologia:",seq_record.annotations["topology"])
print("Referências externas:",seq_record.dbxrefs)
print()
################################

for s in seq_record.features:
    if s.type == "CDS":
        qual = dict(s.qualifiers)
        
        if "gene" in qual:
            if qual["gene"][0] in genes_int:
                loc = s.location
                #########################
                print("Gene:",qual["gene"][0])
                print("Produto:", qual["product"])
                #print("Início:", int(loc.start))
                #print("Fim:", int(loc.end))
                #print("Cadeia:", int(loc.strand))
                #print("ID Proteína:", qual["protein_id"])
                print()
                #########################
                local[qual["gene"][0]] = (int(loc.start), int(loc.end), loc.strand)
                gene_seq[qual["gene"][0]] = s.extract(seq_record.seq)
                prot_id[qual["gene"][0]] = qual["protein_id"]
                prot[qual["gene"][0]] = qual["translation"]

handle.close()



Título: Staphylococcus aureus subsp. aureus NCTC 8325, complete genome
ID GenBank (e outros IDs): ['CP000253']
Taxonomia: ['Bacteria', 'Firmicutes', 'Bacilli', 'Bacillales', 'Staphylococcaceae', 'Staphylococcus']
Tipo de molécula: DNA
Topologia: circular
Referências externas: ['BioProject:PRJNA237', 'BioSample:SAMN02604235']



In [3]:
from Bio import ExPASy
import Bio.SwissProt as sp

with open("prot_ileS.txt") as handle:
    record = sp.read(handle)
    print("Nome:", record.entry_name,"\n")
    for acc in record.accessions:
        print("ID UniProt:", acc)
    print()
    for des in record.description.split(";"):
        print(des)
    print("Organismo:",repr(record.organism),"\n")
    print("Classificao do organismo:",record.organism_classification,"\n")
    print(record.sequence[:20] + "...","\n")
    print("Tamanho da seq:", record.sequence_length,"\n")
    print("Palavras Chave:",record.keywords,"\n")
    
    #############################
    #rec = sp.KeyWList(handle)
    #print("Termos GO:",rec)
    #############################
    
counter = 1
for r in record.features:
    if r.type == "ACT_SITE":
        print(f"Sítio ativo {counter}:",r.location)
        counter += 1
    

Nome: SYI1_STAAU 

ID UniProt: P41972

RecName: Full=Isoleucine--tRNA ligase
 EC=6.1.1.5
 AltName: Full=Isoleucyl-tRNA synthetase
 Short=IleRS

Organismo: 'Staphylococcus aureus.' 

Classificao do organismo: ['Bacteria', 'Firmicutes', 'Bacilli', 'Bacillales', 'Staphylococcaceae', 'Staphylococcus'] 

MDYKETLLMPKTDFPMRGGL... 

Tamanho da seq: 917 

Palavras Chave: ['3D-structure', 'Aminoacyl-tRNA synthetase', 'ATP-binding', 'Cytoplasm', 'Direct protein sequencing', 'Ligase', 'Metal-binding', 'Nucleotide-binding', 'Protein biosynthesis', 'Zinc'] 



In [4]:

from Bio import SeqIO

genes_int = ["mprF", "ileS"]
#local
#gene_seq
#prot_id
#prot


for g in genes_int:
    ####################
    print("Gene:",g)
    print("Início:",local[g][0])
    print("Fim:",local[g][1])
    print("Cadeia:",local[g][2])
    print("Sequência DNA:",gene_seq[g][0:75]+"...")
    print("ID Proteína:",prot_id[g][0])
    print("Sequência Proteica:",prot[g][0][0:75]+"...")
    print("----------")
    ####################
    inicio, fim, strand = local[g]
    annot = ";".join([g,str(inicio),str(fim),str(strand),prot_id[g][0]])
    
    file = open(g+".fasta","w")
    file.write(">"+annot+"\n"+str(gene_seq[g]))
    file.close()
    
    file = open(g+"_"+prot_id[g][0]+".fasta","w")
    file.write(">"+annot+"\n"+prot[g][0])
    file.close()


Gene: mprF


KeyError: 'mprF'